In [6]:
######################################     Modules     #######################################
import sys
from time import process_time
import os

import verbose_module
sys.path.append('../../.')
import pIRPgym

########################     Instance generator and Environment     #########################
# Instance Generator
### pIRP model's parameters
# Stochasticity
stochastic_params = ['d','q']
look_ahead = ['d','q']


# Historical data
historical_data = ['*']


# Other parameters
backorders = 'backorders'

env_config = {  'M':13,'K':15,'T':12,'F':13,'Q':2000,
                'S':6,'LA_horizon':4,
                'd_max':2000,'hist_window':60,
                'back_o_cost':10000
            }

# dCVRP Instances

In [2]:
########################     Instance generator and Environment     #########################
# Creating instance generator object
inst_gen = pIRPgym.instance_generator(look_ahead, stochastic_params,
                              historical_data, backorders, env_config = env_config)

instances = dict()
instances['Li'] = [i for i in os.listdir('/Users/juanbeta/My Drive/Research/Supply Chain Analytics/pIRPenv/pIRPgym/Instances/CVRP Instances/dCVRP/Li') if i[-3:]=='vrp']
instances['Golden'] = [i for i in os.listdir('/Users/juanbeta/My Drive/Research/Supply Chain Analytics/pIRPenv/pIRPgym/Instances/CVRP Instances/dCVRP/Golden') if i[-3:]=='vrp']
instances['Li'].sort();instances['Golden'].sort()

### Environment 
# Creating environment object
routing = True
inventory = False    
perishability = False
env = pIRPgym.steroid_IRP(routing,inventory,perishability)
env.reset(inst_gen)

In [3]:
################################## Policy Evaluation ##################################
''' Parameters '''
verbose = True
show_gap = True

if verbose: verbose_module.routing_instances.print_head(show_gap)

for ss,inst_list in instances.items():
    for instance in inst_list:
        # Upload dCVRP instance
        purchase,benchmark = inst_gen.upload_CVRP_instance(ss, instance)

        if verbose: string = verbose_module.routing_instances.print_inst(ss,instance,benchmark[0],benchmark[1])

        nn_routes,nn_obj,nn_info,nn_time = pIRPgym.Routing.NearestNeighbor(purchase,inst_gen,env.t)                                         # Nearest Neighbor
        if verbose: string = verbose_module.routing_instances.print_routing_update(string,
                                                                    nn_obj,len(nn_routes),nn_time,show_gap,benchmark)
            
        RCL_obj,RCL_veh,RCL_time = pIRPgym.Routing.evaluate_stochastic_policy(pIRPgym.Routing.RCL_Heuristic,
                                                                                           purchase,inst_gen,env,n=30,averages=True,dynamic_p=False)
        if verbose: string = verbose_module.routing_instances.print_routing_update(string,
                                                                    RCL_obj,RCL_veh,RCL_time,show_gap,benchmark,end=True)
        # GA_routes,GA_obj,GA_info,GA_time,_ = pIRPgym.Routing.HybridGenticAlgorithm(purchase,inst_gen,env.t,return_top=False,rd_seed=0,time_limit=30)    # Genetic Algorithm
        # if verbose: string = verbose_module.routing_instances.print_routing_update(string,
        #                                                             GA_obj,len(GA_routes),GA_time,show_gap,benchmark)
        # HyGeSe_routes, HyGeSe_distance, HyGeSe_time  = pIRPgym.Routing.HyGeSe.HyGeSe_routing(purchase,inst_gen,env.t,time_limit=30)                                  # Hybrid Genetic Search (CVRP)
        # if verbose: string = verbose_module.routing_instances.print_routing_update(string,HyGeSe_distance,len(HyGeSe_routes),HyGeSe_time,show_gap,benchmark,True)                                    # Column Generation algorithm
    print('\n')

*************************************  Routing Strategies on Classic Instances  *************************************
--------|	BKS 	|	   NN 	 	|	   RCL 	 	|	   HGA 	 	|	  HGS*
Inst	| #Veh 	 Obj 	| t(s)	 #Veh 	 gap 	| t(s) 	 #Veh 	gap 	| t(s) 	 #Veh 	 gap 	| t(s) 	 #Veh 	 gap 	|
----------------------------------------------------------------------------------------------------------------------


# CVRP Instances

In [7]:
########################     Instance generator and Environment     #########################
# Creating instance generator object
inst_gen = pIRPgym.instance_generator(look_ahead, stochastic_params,
                              historical_data, backorders, env_config = env_config)

instances = dict()
# instances = [i for i in os.listdir('/Users/juanbeta/My Drive/Research/Supply Chain Analytics/pIRPenv/pIRPgym/Instances/CVRP Instances/CVRP/Uchoa') if i[-3:]=='vrp']
instances = [i for i in os.listdir('C:/Users/jm.betancourt/Documents/Research/pIRPgym/pIRPgym/Instances/CVRP Instances/CVRP/Uchoa') if i[-3:]=='vrp']
instances.sort()
instances = instances[1:] + [instances[0]]

### Environment 
# Creating environment object
routing = True
inventory = False    
perishability = False
env = pIRPgym.steroid_IRP(routing,inventory,perishability)
env.reset(inst_gen)

In [8]:
################################## Policy Evaluation ##################################
''' Parameters '''
verbose = True
show_gap = True

ranges = [i for i in range(1,11)]
performance = {'nn':{i:{'FO':list(),'Time':list()} for i in ranges},'RCL':{i:{'FO':list(),'Time':list()} for i in ranges},'GA':{i:{'FO':list(),'Time':list()} for i in ranges}}


if verbose: verbose_module.routing_instances.print_head(show_gap)

for instance in instances:
    # Upload dCVRP instance
    purchase,benchmark = inst_gen.upload_CVRP_instance('Uchoa',instance)

    if verbose: string = verbose_module.routing_instances.print_inst('Uchoa',instance,benchmark[0],benchmark[1])

    nn_routes,nn_obj,nn_info,nn_time = pIRPgym.Routing.NearestNeighbor(purchase,inst_gen,env.t)                                         # Nearest Neighbor
    if verbose: string = verbose_module.routing_instances.print_routing_update(string,
                                                                nn_obj,len(nn_routes),nn_time,show_gap,benchmark)
    performance['nn'][len(purchase)//100]['FO'].append(nn_obj);performance['nn'][len(purchase)//100]['Time'].append(nn_time)


    RCL_obj,RCL_veh,RCL_time = pIRPgym.Routing.evaluate_stochastic_policy(pIRPgym.Routing.RCL_Heuristic,
                                                                                        purchase,inst_gen,env,n=30,averages=True,dynamic_p=False)
    if verbose: string = verbose_module.routing_instances.print_routing_update(string,
                                                                RCL_obj,RCL_veh,RCL_time,show_gap,benchmark)
    performance['RCL'][len(purchase)//100]['FO'].append(RCL_obj);performance['RCL'][len(purchase)//100]['Time'].append(RCL_time)

    GA_routes,GA_obj,GA_info,GA_time,_ = pIRPgym.Routing.HybridGenticAlgorithm(purchase,inst_gen,env.t,return_top=False,rd_seed=0,time_limit=30)    # Genetic Algorithm
    if verbose: string = verbose_module.routing_instances.print_routing_update(string,
                                                                GA_obj,len(GA_routes),GA_time,show_gap,benchmark,end=True)
    performance['GA'][len(purchase)//100]['FO'].append(GA_obj);performance['GA'][len(purchase)//100]['Time'].append(GA_time)
    # HyGeSe_routes, HyGeSe_distance, HyGeSe_time  = pIRPgym.Routing.HyGeSe.HyGeSe_routing(purchase,inst_gen,env.t,time_limit=30)                                  # Hybrid Genetic Search (CVRP)
    # if verbose: string = verbose_module.routing_instances.print_routing_update(string,HyGeSe_distance,len(HyGeSe_routes),HyGeSe_time,show_gap,benchmark,True)                                    # Column Generation algorithm
    

*************************************  Routing Strategies on Classic Instances  *************************************
--------|	BKS 	|	   NN 	 	|	   RCL 	 	|	   HGA 	 	|	  HGS*
Inst	| #Veh 	 Obj 	| t(s)	 #Veh 	 gap 	| t(s) 	 #Veh 	gap 	| t(s) 	 #Veh 	 gap 	| t(s) 	 #Veh 	 gap 	|
----------------------------------------------------------------------------------------------------------------------
n101-k25| 25 	27591 	| 0.00	  26	 52.02	| 0.00	  26.73	 62.52	| 9.91	  26	 32.46	|
n106-k14| 14 	26362 	| 0.00	  14	 9.61	| 0.00	  14.0	 40.48	| 9.17	  14	 6.81	|
n110-k13| 13 	14971 	| 0.00	  13	 28.86	| 0.00	  13.0	 116.25	| 9.06	  13	 19.07	|
n115-k10| 10 	12747 	| 0.00	  10	 51.64	| 0.00	  10.0	 170.78	| 9.23	  10	 33.95	|
n120-k6| 6 	13332 	| 0.00	  6	 19.18	| 0.00	  6.0	 150.59	| 9.41	  6	 14.09	|
n125-k30| 30 	55539 	| 0.00	  32	 23.07	| 0.00	  31.63	 33.94	| 10.80	  31	 17.2	|
n129-k18| 18 	28940 	| 0.00	  18	 24.24	| 0.00	  18.0	 73.48	| 9.36	  18	 18.5	|
n134-k13| 13 	10916 	| 0.00	  

KeyError: 'X-n876-k59'

In [4]:
performance

{'nn': {1: {'FO': [41944,
    28896,
    19292,
    19330,
    15889,
    68354,
    35956,
    16526,
    16845,
    22533,
    56663],
   'Time': [0.0029360000000000497,
    0.001961999999998909,
    0.002072000000005403,
    0.002255000000005225,
    0.0038070000000089976,
    0.0040180000000020755,
    0.0030920000000094205,
    0.0034269999999878564,
    0.0031800000000146156,
    0.003765999999984615,
    0.0035579999999981737]},
  2: {'FO': [], 'Time': []},
  3: {'FO': [], 'Time': []},
  4: {'FO': [], 'Time': []},
  5: {'FO': [], 'Time': []},
  6: {'FO': [], 'Time': []},
  7: {'FO': [], 'Time': []},
  8: {'FO': [], 'Time': []},
  9: {'FO': [], 'Time': []},
  10: {'FO': [], 'Time': []}},
 'RCL': {1: {'FO': [44840.7,
    37033.5,
    32375.4,
    34516.566666666666,
    33408.166666666664,
    74388.0,
    50204.23333333333,
    28083.033333333333,
    36647.8,
    43443.8,
    68382.66666666667],
   'Time': [0.01014006666666667,
    0.011693366666667041,
    0.012584066666666161,

In [5]:
'X-n1001-k43' in instances

False

In [7]:
'X-n1001-k43.vrp' in instances

True